### Objectives
The purpose of this notebook is to develop a solid understanding of the variables in the dataset and their relationship to one another in order to guide ML model development. Inisghts will help steer which types of models can best address the overall project objetive as well as the additional preprocessing required to have a dataset which is ready for ML application.

As a secondary objective, the descriptive analysis can be used to test assumptions held within the business and draw out preliminary insights. 

### Insights of Interest
(Insert the most important insights, create interactive plotly plots for these select few)

In [ ]:
# imports (pandas, plotly)

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import sys
import os

In [5]:
df = pd.read_excel("./data/cleaned_data.xlsx")

In [11]:
print(df.duplicated().sum()) # no duplicates. add to data cleaning notebook after. 

0


### Preparation

In [14]:
df.columns

Index(['id', 'name', 'goal', 'pledged', 'state', 'disable_communication',
       'country', 'currency', 'deadline', 'state_changed_at', 'created_at',
       'launched_at', 'staff_pick', 'backers_count', 'static_usd_rate',
       'usd_pledged', 'category', 'spotlight', 'name_len', 'name_len_clean',
       'blurb_len', 'blurb_len_clean', 'deadline_weekday',
       'state_changed_at_weekday', 'created_at_weekday', 'launched_at_weekday',
       'deadline_month', 'deadline_day', 'deadline_yr', 'deadline_hr',
       'state_changed_at_month', 'state_changed_at_day', 'state_changed_at_yr',
       'state_changed_at_hr', 'created_at_month', 'created_at_day',
       'created_at_yr', 'created_at_hr', 'launched_at_month',
       'launched_at_day', 'launched_at_yr', 'launched_at_hr',
       'create_to_launch_days', 'launch_to_deadline_days',
       'launch_to_state_change_days', 'usd_goal', 'name2', 'name_len2',
       'name_len_clean_2'],
      dtype='object')

In [17]:
# checking if currency always matches with country
df['cc'] = df['country'] + df['currency']

In [19]:
df['cc'].unique() # currency for fundraising does not differ in the same country. no extra information captured having both the country and currency variable

array(['GBGBP', 'DEEUR', 'USUSD', 'AUAUD', 'CACAD', 'NONOK', 'FREUR',
       'BEEUR', 'NZNZD', 'ITEUR', 'SESEK', 'IEEUR', 'DKDKK', 'ESEUR',
       'NLEUR', 'CHCHF', 'ATEUR', 'LUEUR'], dtype=object)

In [20]:
# dropping unnecessary columns
df = df.drop(columns=['id', 'goal', 'pledged', 'currency', 'cc', 'deadline', 'state_changed_at','created_at','launched_at', 'static_usd_rate' ], axis=1)
# dropping id col
# non-US currency cols
# timestamps since they've already been broken down into individual variables

In [32]:
# and removing the ones i remade. 
df = df.drop(columns=['name', 'name_len', 'name_len_clean'], axis=1)


In [35]:
catcols = df.select_dtypes(include=['object', 'bool']).columns.to_list()

In [38]:
numcols = [col for col in df.columns if col not in catcols]

### Univariate Analysis

In [42]:
# descriptive statistics on numerical variables
df.describe().round(2)

,backers_count,usd_pledged,blurb_len,blurb_len_clean,deadline_month,deadline_day,deadline_yr,deadline_hr,state_changed_at_month,state_changed_at_day,...,launched_at_month,launched_at_day,launched_at_yr,launched_at_hr,create_to_launch_days,launch_to_deadline_days,launch_to_state_change_days,usd_goal,name_len2,name_len_clean_2
count,15474.00,15474.00,15470.00,15470.00,15474.00,15474.00,15474.00,15474.00,15474.00,15474.00,...,15474.00,15474.00,15474.00,15474.00,15474.00,15474.00,15474.00,1.547400e+04,15474.00,15474.00
mean,181.17,19975.49,18.99,13.03,6.78,15.66,2014.37,13.05,6.78,15.65,...,6.62,15.30,2014.29,12.59,46.91,34.68,32.02,9.474142e+04,5.80,4.73
std,1287.83,114439.55,4.64,3.26,3.42,9.04,1.12,6.06,3.41,8.99,...,3.36,8.78,1.12,5.59,101.77,11.98,13.67,1.462852e+06,2.84,2.25
min,0.00,0.00,1.00,1.00,1.00,1.00,2009.00,0.00,1.00,1.00,...,1.00,1.00,2009.00,0.00,0.00,1.00,0.00,1.000000e+00,1.00,0.00
25%,2.00,24.86,17.00,11.00,4.00,8.00,2014.00,9.00,4.00,8.00,...,4.00,8.00,2014.00,9.00,3.00,30.00,29.00,4.000000e+03,3.00,3.00
50%,12.00,676.84,20.00,13.00,7.00,15.00,2015.00,13.00,7.00,15.00,...,7.00,15.00,2015.00,13.00,13.00,30.00,30.00,1.250000e+04,6.00,5.00
75%,63.00,5896.88,22.00,15.00,10.00,23.00,2015.00,18.00,10.00,23.00,...,10.00,23.00,2015.00,17.00,43.00,40.00,35.00,4.482050e+04,8.00,6.00
max,105857.00,6225354.98,35.00,30.00,12.00,31.00,2016.00,23.00,12.00,31.00,...,12.00,31.00,2016.00,23.00,1528.00,91.00,91.00,1.000000e+08,16.00,14.00


**Observations (First Pass)**
* Zero minimums for backers count and usd pledged makes sense since some may have been cancelled/ just started / suspended. For analysis, could help to limit to only records which were not suspended + those who had been running for at least a quarter of their expected run time by the time of data extraction. Potentially only those that completed their run or were cancelled. 
* 25% to 75% fairly close together for these two variables as well but the max is very far from that. Check distribution & check with SMRs for if these should be considered outliers or if there's an expected portion of projects that would have those very high numbers



In [ ]:
# histogram and boxplot distributions of numerical variables

In [ ]:
# check number of unique values for categorical variables

In [ ]:
# frequency distributions of categrocial variables

In [ ]:
# check proportion assigned to each of the top 10 values for the categorical variables

In [ ]:
# check nature of categories for sense of how to encode 

### Bivariate Analysis

In [ ]:
# create version of dataset with encoded categoricals and normalized numerical, perform 2 component PCA and plot for an initial idea of whether there are clusters present
# make commentary, but don't abandon clustering yet

In [ ]:
# pairplots for all the numerical variables, see if any relationships to observe. 
# especially among variables which seemed to have meaningful divides in the univariate analysis

### Multivariate

In [ ]:
# correlation martrix for numerical
# chi-square tests for categorical

In [ ]:
# check proportion of each categorical variable value that is in each state (mosaic plot)

In [ ]:
# interesting bivariate plots from earlier color coded by state

### Multivariate Analysis